In [1]:
from diffusers import StableDiffusionPipeline
import torch

# 학습 결과물이 저장된 경로
model_path = "minhwa_textual_inversion"

# 스테이블 디퓨전 파이프라인 불러오기
pipeline = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16
).to("cuda")

# 학습된 '민화 스타일' 임베딩 불러오기
pipeline.load_textual_inversion(model_path)

/workspace/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 6/6 [00:11<00:00,  1.83s/it]
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [2]:
# 프롬프트에 placeholder_token (<minhwa-style>)을 사용하여 이미지 생성
prompt = "A korean tiger in the style of <minhwa-style>"
image = pipeline(prompt, num_inference_steps=50).images[0]

# 이미지 저장
image.save("minhwa_style_tiger.png")

100%|██████████| 50/50 [00:04<00:00, 11.99it/s]


In [3]:
# 프롬프트에 placeholder_token (<minhwa-style>)을 사용하여 이미지 생성
prompt = "A korean tiger"
image = pipeline(prompt, num_inference_steps=50).images[0]

# 이미지 저장
image.save("no_style_tiger.png")

100%|██████████| 50/50 [00:03<00:00, 12.85it/s]


### test

In [1]:
from diffusers import StableDiffusionPipeline
import torch

# 스테이블 디퓨전 파이프라인 불러오기
pipeline = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

/workspace/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 7/7 [00:03<00:00,  1.83it/s]


In [ ]:
# 프롬프트에 placeholder_token (<minhwa-style>)을 사용하여 이미지 생성
prompt = "A korean tiger in the style of <minhwa-style>"
image = pipeline(prompt, num_inference_steps=50).images[0]

# 이미지 저장
image.save("minhwa_style_tiger_notrain.png")

In [ ]:
# 프롬프트에 placeholder_token (<minhwa-style>)을 사용하여 이미지 생성
prompt = "A korean tiger"
image = pipeline(prompt, num_inference_steps=50).images[0]

# 이미지 저장
image.save("no_style_tiger_notrain.png")

## lora inference

In [9]:
from diffusers import StableDiffusionPipeline
import torch

# 1. LoRA 모델 가중치가 저장된 경로 (학습 시 --output_dir 경로와 동일)
lora_model_path = "minhwa_lora_model/checkpoint-1500"

# 2. 베이스 모델 경로 (매우 중요! 학습 시 --pretrained_model_name_or_path 경로와 동일)
base_model_path = "runwayml/stable-diffusion-v1-5"

# 3. 스테이블 디퓨전 파이프라인 불러오기 (반드시 학습에 사용한 베이스 모델로!)
pipeline = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16
)

# 4. LoRA 가중치 불러오기 및 파이프라인에 적용
# .bin 또는 .safetensors 형식의 LoRA 가중치 파일을 자동으로 찾아서 로드합니다.
pipeline.load_lora_weights(lora_model_path)

# 5. GPU로 모델 이동
pipeline.to("cuda")

# 6. 프롬프트 작성 (LoRA 방식)
# <special-token> 없이, 학습 데이터의 특징을 설명하는 단어들을 조합합니다.
# 학습용 metadata.jsonl에 넣었던 "a minhwa painting", "sansudo", "tiger" 같은 키워드를 활용하면 효과적입니다.
prompt = "a minhwa painting of a tiger in a landscape, sansudo style"
negative_prompt = "blurry, worst quality, low quality" # 필요 시 부정적 프롬프트 추가

# 7. 이미지 생성
# guidance_scale은 프롬프트를 얼마나 따를지 정하는 값으로, 7~8 사이가 안정적입니다.
image = pipeline(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=50,
    guidance_scale=7.5
).images[0]

# 8. 이미지 저장
image.save("minhwa_lora_tiger_checkpoint-1500.png")

Loading pipeline components...: 100%|██████████| 7/7 [00:14<00:00,  2.10s/it]
No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
100%|██████████| 50/50 [00:02<00:00, 20.80it/s]


In [10]:
# "tiger"(호랑이)를 "rabbit"(토끼)으로 변경
prompt = "a minhwa painting of a rabbit"
negative_prompt = "blurry, worst quality, low quality" # 필요 시 부정적 프롬프트 추가

# 7. 이미지 생성
# guidance_scale은 프롬프트를 얼마나 따를지 정하는 값으로, 7~8 사이가 안정적입니다.
image = pipeline(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=50,
    guidance_scale=7.5
).images[0]

# 8. 이미지 저장 - *** 저장 파일 이름을 변경합니다 ***
image.save("minhwa_lora_rabbit_checkpoint-1500.png")


 16%|█▌        | 8/50 [00:00<00:02, 19.54it/s]

100%|██████████| 50/50 [00:02<00:00, 20.11it/s]


### lora with category

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

# 1. LoRA 모델 가중치가 저장된 경로 (학습 시 --output_dir 경로와 동일)
lora_model_path = "minhwa_lora_model_withcategory_r32"

# 2. 베이스 모델 경로 (매우 중요! 학습 시 --pretrained_model_name_or_path 경로와 동일)
base_model_path = "runwayml/stable-diffusion-v1-5"

# 3. 스테이블 디퓨전 파이프라인 불러오기 (반드시 학습에 사용한 베이스 모델로!)
pipeline = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16
)

# 4. LoRA 가중치 불러오기 및 파이프라인에 적용
# .bin 또는 .safetensors 형식의 LoRA 가중치 파일을 자동으로 찾아서 로드합니다.
pipeline.load_lora_weights(lora_model_path)

# 5. GPU로 모델 이동
pipeline.to("cuda")

# 6. 프롬프트 작성 (LoRA 방식)
# <special-token> 없이, 학습 데이터의 특징을 설명하는 단어들을 조합합니다.
# 학습용 metadata.jsonl에 넣었던 "a minhwa painting", "sansudo", "tiger" 같은 키워드를 활용하면 효과적입니다.
# prompt = "a minhwa painting of a tiger in a landscape, sansudo style"
prompt = "a minhwa painting, HODO, Tiger, in a landscape."
negative_prompt = "blurry, worst quality, low quality" # 필요 시 부정적 프롬프트 추가

# 7. 이미지 생성
# guidance_scale은 프롬프트를 얼마나 따를지 정하는 값으로, 7~8 사이가 안정적입니다.
image = pipeline(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=50,
    guidance_scale=7.5
).images[0]

# 8. 이미지 저장
image.save("minhwa_lora_tiger_withcategory_r32_prompting.png")

Loading pipeline components...: 100%|██████████| 7/7 [00:14<00:00,  2.08s/it]
No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
 52%|█████▏    | 26/50 [00:01<00:01, 21.30it/s]

In [ ]:
import os
from diffusers import StableDiffusionPipeline
import torch

# --- 설정값 ---

# 1. 체크포인트들이 저장된 기본 폴더 경로
# 학습 명령어의 --output_dir 경로와 동일해야 합니다.
base_output_dir = "minhwa_lora_model_withcategory_r32" 

# 2. 베이스 모델 경로 (학습 시 사용한 모델과 동일)
base_model_path = "runwayml/stable-diffusion-v1-5"

# 3. 생성할 프롬프트 정보
prompt = "a minhwa painting of a tiger in a landscape, sansudo style"
negative_prompt = "blurry, worst quality, low quality"

# 4. 반복할 에폭 및 스텝 정보
num_epochs = 15
steps_per_epoch = 162

# --- 설정 끝 ---


# 5. 스테이블 디퓨전 파이프라인을 한 번만 불러옵니다. (효율성)
print("Loading base pipeline...")
pipeline = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16
)
pipeline.to("cuda")
print("Pipeline loaded.")

# 6. 각 에폭별 체크포인트에 대해 루프를 실행합니다.
for epoch in range(1, num_epochs + 1):
    # 현재 에폭에 해당하는 글로벌 스텝 계산
    global_step = epoch * steps_per_epoch
    
    # 로드할 LoRA 가중치 경로 구성
    lora_model_path = os.path.join(base_output_dir, f"checkpoint-{global_step}")
    
    # 생성될 이미지 파일 이름 구성
    output_image_name = f"minhwa_lora_tiger_withcategory_r32_prompting_{epoch}.png"
    
    print(f"\n--- Processing Epoch {epoch}/{num_epochs} (Checkpoint: {global_step}) ---")

    # 체크포인트 폴더가 실제로 존재하는지 확인
    if not os.path.exists(lora_model_path):
        print(f"Warning: Checkpoint directory not found, skipping: {lora_model_path}")
        continue # 다음 루프로 넘어감

    try:
        # 7. LoRA 가중치 불러오기 및 파이프라인에 적용
        # load_lora_weights는 내부적으로 이전 LoRA 가중치를 해제하고 새로 로드합니다.
        pipeline.load_lora_weights(lora_model_path)
        print(f"Successfully loaded LoRA weights from {lora_model_path}")

        # 8. 이미지 생성
        image = pipeline(
            prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=50,
            guidance_scale=7.5
        ).images[0]
        
        # 9. 이미지 저장
        image.save(output_image_name)
        print(f"Image saved successfully: {output_image_name}")

    except Exception as e:
        print(f"An error occurred while processing {lora_model_path}: {e}")


print("\nAll epochs processed. Image generation complete.")

Loading pipeline components...:  29%|██▊       | 2/7 [00:02<00:05,  1.11s/it]


KeyboardInterrupt: 